In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
from scipy.linalg import eigh
from scipy import sparse
from node_embedding_attack.utils import *
from node_embedding_attack.embedding import *
from node_embedding_attack.perturbation_attack import *

### Load and preprocess the data

In [2]:
def load_cora_orig(data_dir, largest_cc=False):
    """Only loads the graph structure ignoring node features"""
    g_nx = nx.read_edgelist(path=os.path.expanduser(os.path.join(data_dir, "cora.cites")))
    
    for edge in g_nx.edges(data=True):
        edge[2]['label'] = 'cites'

    if largest_cc:
        # Select the largest connected component. For clarity we ignore isolated
        # nodes and subgraphs; having these in the data does not prevent the
        # algorithm from running and producing valid results.
        g_nx_ccs = (g_nx.subgraph(c).copy() for c in nx.connected_components(g_nx))
        g_nx = max(g_nx_ccs, key=len)
        print("Largest subgraph statistics: {} nodes, {} edges".format(
            g_nx.number_of_nodes(), g_nx.number_of_edges()))   

    adj_matrix = nx.to_numpy_array(g_nx)
    adj_matrix = sparse.csr_matrix(adj_matrix)
        
    return g_nx, adj_matrix

In [3]:
def load_cora(data_dir, largest_cc=False):
    """Only loads the graph structure ignoring node features"""
    path = os.path.expanduser(os.path.join(data_dir, 
                                           "ind.cora.graph"))

    with open(path, "rb") as f:
        G_dict = pkl.load(f, encoding="latin1")
    
    G = nx.from_dict_of_lists(G_dict)
    
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix = sparse.csr_matrix(adj_matrix)

    return G, adj_matrix        

In [4]:
def load_citeseer_old(data_home):
    """Only returns the graph structure ignoring node features"""

    df = pd.read_csv(
        os.path.join(data_home, "citeseer.content"),
        sep=r"\s+",
        header=None,
        index_col=0,
    )
    df.index = df.index.map(str)

    features_df = df.iloc[:, :-1]
    labels_df = df.iloc[:, -1]

    edge_list_df = pd.read_csv(
        os.path.join(data_home, "citeseer.cites"), sep=r"\s+", dtype=str, header=None
    )

    idx_map = {j: i for i, j in enumerate(df.index)}

    H = nx.from_pandas_edgelist(edge_list_df, 0, 1)
    G = nx.relabel.relabel_nodes(H, idx_map)

    # This dataset has about 15 nodes in the edge list that don't have corresponding entries
    # in citeseer.content, that is don't have features. We need to identify them and then remove
    # them from the graph along with all the edges to/from them.
    nodes_to_remove = [n for n in G.nodes() if type(n) == str]
    print(f"Removing {len(nodes_to_remove)} nodes from graph")
    G.remove_nodes_from(nodes_to_remove)

    adj_matrix = nx.to_numpy_array(G)
    adj_matrix = sparse.csr_matrix(adj_matrix)

    #adj_matrix = nx.to_scipy_sparse_matrix(G, nodelist=sorted(G.nodes()), format="coo")

    return G, adj_matrix


In [5]:
def load_citeseer(data_home):
    """Only returns the graph structure ignoring node features"""

    path = os.path.expanduser(os.path.join(data_home, 
                                           "ind.citeseer.graph"))

    with open(path, "rb") as f:
        G_dict = pkl.load(f, encoding="latin1")
    
    G = nx.from_dict_of_lists(G_dict)
    
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix = sparse.csr_matrix(adj_matrix)

    return G, adj_matrix

In [6]:
def load_polblogs(data_home=None):
    graph = load_dataset('data/polblogs.npz')
    adj_matrix = graph['adj_matrix']
    labels = graph['labels']

    adj_matrix, labels = standardize(adj_matrix, labels)
    adj_matrix = np.asarray(adj_matrix.todense())
    G = nx.from_numpy_matrix(adj_matrix)
    adj_matrix = sparse.csr_matrix(adj_matrix)
    
    # write the graph and labels to disk
    nx.write_gpickle(G, "polblogs_graph.gpickle")
    np.save("polblogs_labels.npy", labels)
    
    return G, adj_matrix

In [ ]:
def load_pubmed(data_home):
    """Only returns the graph structure ignoring node features"""

    path = os.path.expanduser(os.path.join(data_home, 
                                           "ind.pubmed.graph"))

    with open(path, "rb") as f:
        G_dict = pkl.load(f, encoding="latin1")
    
    G = nx.from_dict_of_lists(G_dict)
    
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix = sparse.csr_matrix(adj_matrix)

    return G, adj_matrix

### Select the dataset

In [16]:
dataset = "polblogs"  # one of cora, citeseer, polblogs, pubmed
load_data = load_cora
if dataset=="cora":
    data_dir = "~/data/cora"
elif dataset == "citeseer":
    data_dir = "~/data/citeseer/"
    load_data = load_citeseer
elif dataset == "polblogs":
    data_dir = None
    load_data = load_polblogs
elif dataset == "pubmed":
    data_dir = "~/data/pubmed/"
    load_data = load_citeseer

In [17]:
g_nx, adj_matrix = load_data(data_dir)
n_nodes = adj_matrix.shape[0]
# The example code selects the largest component but we won't do that
# here. The question is, does this effect the quality of attacks?

In [18]:
adj_matrix.shape, n_nodes, g_nx.number_of_edges()

((1222, 1222), 1222, 16717)

In [19]:
type(adj_matrix)

scipy.sparse.csr.csr_matrix

In [20]:
nx.is_connected(g_nx)

True

### Set hyperparameters

In [94]:
n_flips = 1000
dim = 32
window_size = 5
#attack_method = "add"

### Store attacked graph

In [95]:
def attack_graph(adj_matrix, n_flips, dim, 
                 window_size, seed=0, method="add"):
    """Method for attacking a graph by adding or removing edges"""
    if method=="add":
        candidates = generate_candidates_addition(adj_matrix=adj_matrix,
                                                  n_candidates=n_flips, 
                                                  seed=seed)
    else:
        candidates = generate_candidates_removal(adj_matrix=adj_matrix, 
                                                 seed=seed)
        
    our_flips = perturbation_top_flips(adj_matrix, 
                                       candidates, 
                                       n_flips, 
                                       dim, 
                                       window_size)
    #
    A = np.array(adj_matrix.todense())
    
    A_flipped = A.copy()
    A_flipped[our_flips[:, 0], our_flips[:, 1]] = 1 - A[our_flips[:, 0], our_flips[:, 1]]
    A_flipped[our_flips[:, 1], our_flips[:, 0]] = 1 - A[our_flips[:, 1], our_flips[:, 0]]
    
    return A_flipped  # The attacked adjacency matrix

### Save attacked graph to disk

In [96]:
ele = 'attack'
#corrupted_A = corrupt_adjacency(A, ele, l)
dir_name = os.path.join("attacked_datasets", 
                        dataset, 
                        ele)
print(dir_name)
i = 1

attacked_datasets/polblogs/attack


In [97]:
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

In [98]:
seeds = [0, 42, 10003, 918358, 63947, 9123, 43218, 90, 300134, 1239876]
if dataset=="cora":
    num_flips = [ -2000, -1000, -500, 500, 1000, 2000, 5000 ]
elif dataset=="citeseer":
    num_flips = [ -2000, -1000, -500, 500, 1000, 2000, 5000 ]
elif dataset=="polblogs":
    num_flips = [ -2000, -1000, -500, 500, 1000, 2000, 5000 ]
elif dataset=="pubmed":
    num_flips = [ -20000, -10000, -5000, -2000, 2000, 5000, 10000, 20000, 40000 ]

In [99]:
len(seeds)

10

In [100]:
for n_flips in num_flips:
    print(f"Calculating for n_flips={n_flips}")
    for i, seed in enumerate(seeds):
        if n_flips < 0:
            method = "remove"
            n_flips_ = -n_flips
        else:
            n_flips_ = n_flips
            method = "add"
        A_flipped = attack_graph(adj_matrix=adj_matrix, 
                                 n_flips=n_flips_, 
                                 dim=dim, 
                                 window_size=window_size, 
                                 method=method, 
                                 seed=seed)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        file_name = dataset + "_" + ele + "_"+method+"_"+str(n_flips_)+"_v_"+str(i)
        print(f"file_name: {file_name}")
        #np.save(os.path.join(dir_name,file_name), A_flipped)

        graph = nx.from_numpy_array(A_flipped)
        file_name += ".gpickle"
        nx.write_gpickle(graph, os.path.join(dir_name, file_name))

Calculating for n_flips=-2000
file_name: polblogs_attack_remove_2000_v_0
file_name: polblogs_attack_remove_2000_v_1
file_name: polblogs_attack_remove_2000_v_2
file_name: polblogs_attack_remove_2000_v_3
file_name: polblogs_attack_remove_2000_v_4
file_name: polblogs_attack_remove_2000_v_5
file_name: polblogs_attack_remove_2000_v_6
file_name: polblogs_attack_remove_2000_v_7
file_name: polblogs_attack_remove_2000_v_8
file_name: polblogs_attack_remove_2000_v_9
Calculating for n_flips=-1000
file_name: polblogs_attack_remove_1000_v_0
file_name: polblogs_attack_remove_1000_v_1
file_name: polblogs_attack_remove_1000_v_2
file_name: polblogs_attack_remove_1000_v_3
file_name: polblogs_attack_remove_1000_v_4
file_name: polblogs_attack_remove_1000_v_5
file_name: polblogs_attack_remove_1000_v_6
file_name: polblogs_attack_remove_1000_v_7
file_name: polblogs_attack_remove_1000_v_8
file_name: polblogs_attack_remove_1000_v_9
Calculating for n_flips=-500
file_name: polblogs_attack_remove_500_v_0
file_name

### Random Seed

Effect of seed on attacked graph

In [101]:
n_flips = 10

In [102]:
a_flipped_1 = attack_graph(adj_matrix, 
                           n_flips, 
                           dim, 
                           window_size, seed=0, method="add")

a_flipped_2 = attack_graph(adj_matrix, 
                           n_flips, 
                           dim, 
                           window_size, seed=42, method="add")

In [103]:
n_flips

10

In [104]:
np.sum(a_flipped_1 == 1), np.sum(a_flipped_2 == 1)

(33321, 33321)

In [105]:
g_nx_1 = nx.convert_matrix.from_numpy_array(a_flipped_1)
g_nx_2 = nx.convert_matrix.from_numpy_array(a_flipped_2)

In [106]:
# It should be g_nx.number_of_edges() + n_flips
g_nx_1.number_of_edges(), g_nx_2.number_of_edges()

(16727, 16727)

In [107]:
# Are the two adjacency matrix symmetric? If so, then the difference
# between the matrix and its transpose should be 0
np.sum(a_flipped_1-a_flipped_1.T), np.sum(a_flipped_2-a_flipped_2.T)

(0.0, 0.0)

In [108]:
# If 0 then the two matrices are exactly the same.
np.sum((a_flipped_1-a_flipped_2) != 0)
#np.sum(np.abs(a_flipped_1-a_flipped_2))

40

In [109]:
a_flipped_1[a_flipped_1 != a_flipped_2]

array([1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 1.], dtype=float32)

In [110]:
a_flipped_2[a_flipped_1 != a_flipped_2]

array([0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 0.], dtype=float32)

In [111]:
print(nx.__version__)

2.2


In [112]:
print(np.__version__)

1.16.2
